In [2]:
from numpy.random import seed
seed(1)

from PatientSet import PatientSet
from Constants import Constants
import Metrics
from analysis import *
from Models import *
from copy import copy
import numpy as np
import pandas as pd

from time import time
from datetime import datetime

from dependencies.NCA import NeighborhoodComponentsAnalysis
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.special import softmax
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer

from sklearn.naive_bayes import BernoulliNB, ComplementNB, GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

from imblearn import under_sampling, over_sampling, combine

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

baseline_feature_file = 'data/baselineClustering.csv'

W0925 18:08:04.921000  3688 deprecation_wrapper.py:119] From F:\Skool\EVL_Research\CAMP-RT\PYTHON\PatientSet.py:8: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

Using TensorFlow backend.
F:\Anaconda\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
F:\Anaconda\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [20]:
class MetricLearningClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, n_components = 'auto',
                 random_state = 1,
                 resampler = None,
                 resample_after_nca = True,
                 use_softmax = True):
        self.n_components = n_components
        if n_components is not 'auto':
            self.transformer = NeighborhoodComponentsAnalysis(n_components = n_components)
        self.group_parameters = namedtuple('group_parameters', ['means', 'inv_covariance', 'max_dist'])
        self.resampler = resampler
        self.use_softmax = use_softmax
        self.resample_after_nca = resample_after_nca

    def get_optimal_components(self, x, y):
        n_components = x.shape[1]
        def get_score():
            nca = NeighborhoodComponentsAnalysis(n_components = n_components)
            nca.fit(x,y)
            return silhouette_score(nca.transform(x), y), nca
        score, nca = get_score()
        while True:
            if n_components <= 2:
                return nca
            n_components -= 1
            new_score, new_nca = get_score()
            if new_score > 1.1*score:
                score = new_score
                nca = new_nca
            else:
                return nca
            
    def resample(self, x, y):
        if self.resampler is not None:
            xtemp, ytemp = self.resampler.fit_resample(x,y)
            if len(np.unique(ytemp)) == len(np.unique(y)):
                x = xtemp
                y = ytemp
        return x, y

    def fit(self, x, y):
        if not self.resample_after_nca:
            x, y = self.resample(x,y)
        if self.n_components == 'auto':
            self.transformer = self.get_optimal_components(x, y)
        self.transformer.fit(x, y)
        self.groups = OrderedDict()
        if self.resample_after_nca:
            x, y = self.resample(x,y)
        for group in np.unique(y):
            self.groups[group] = self.group_params(x, y, group)

    def group_params(self, x, y, group):
        targets = np.argwhere(y == group).ravel()
        x_target = self.transformer.transform(x[targets])
        fmeans = x_target.mean(axis = 0)
        inv_cov = np.linalg.pinv(np.cov(x_target.T))
        train_dists = self.mahalanobis_distances(x, self.group_parameters(fmeans, inv_cov, 0))
        parameters = self.group_parameters(fmeans, inv_cov, train_dists.max())
        return parameters

    def mahalanobis_distances(self, x, group):
        x_offset = self.transformer.transform(x) - group.means
        left_term = np.dot(x_offset, group.inv_covariance)
        mahalanobis = np.dot(left_term, x_offset.T).diagonal()
        return mahalanobis

    def predict_proba(self, x):
        all_distances = []
        for group_id, group_params in self.groups.items():
            distances = self.mahalanobis_distances(x, group_params)
            proximity = np.clip(1 - (distances/group_params.max_dist), 0.00001, 1)
            all_distances.append(proximity)
        output = np.hstack(all_distances).reshape(-1, len(self.groups.keys()))
        if self.use_softmax:
            output = softmax(output)
        else:
            output = output/output.sum(axis = 1).reshape(-1,1)
        return output

    def predict(self, x):
        labels = list(self.groups.keys())
        probs = self.predict_proba(self, x)
        max_probs =  np.argmax(probs, axis = 1).ravel()
        ypred = np.zeros(max_probs.shape).astype(np.dtype(labels[0]))
        for i in range(max_probs.shape[0]):
            ypred[i] = labels[max_probs[i]]
        return ypred[i]

    def fit_predict(self, x, y):
        self.fit(x,y)
        return self.predict(x)

class BayesWrapper(BaseEstimator, ClassifierMixin):

    def __init__(self, bayes = BernoulliNB(alpha = 0), n_categories = None):
        if n_categories is None:
            self.encoder = OneHotEncoder(categories = 'auto',
                                         sparse = False,
                                         handle_unknown = 'ignore')
        else:
            self.encoder = KBinsDiscretizer(n_bins = n_categories, encode = 'ordinal')
        self.bayes = bayes

    def fit(self, x, y):
        x = self.encoder.fit_transform(x)
        self.bayes.fit(x,y)
        return self

    def predict(self, x):
        xpred = self.encoder.transform(x)
        return self.bayes.predict(xpred)

    def predict_proba(self, x):
        xpred = self.encoder.transform(x)
        return self.bayes.predict_proba(xpred)

    def fit_predict(self, x, y, probability = False):
        self.fit(x,y)
        if probability:
            return self.predict_proba(x)
        return self.predict(x)

In [88]:
def presplit_roc_cv(classifier, data_split):
    ypred = np.zeros((len(data_split),))
    y = np.array([split['ytest'] for split in data_split])
    i = 0
    for split in data_split:
        classifier.fit(split['xtrain'], split['ytrain'])
        ypred[i] = classifier.predict_proba(split['xtest'])[0,1]
        if i == 0:
            has_importances = hasattr(classifier, 'feature_importances_')
        if has_importances:
            if i == 0:
                importances = classifier.feature_importances_
            else:
                importances += classifier.feature_importances_
        i += 1
    if has_importances:
        importances /= i
        importances = pd.Series(data = importances, index = data_split[0]['feature_labels'])
    else:
        importances = None
    return roc_auc_score(y, ypred), importances

def discretize_continuous_fields(df, n_bins):
    encoder = KBinsDiscretizer(n_bins = n_bins, encode = 'ordinal')
    for col in df.columns:
        vals = df[col].values
        if len(np.unique(vals)) > n_bins:
            df[col] = encoder.fit_transform(vals.reshape(-1,1)).ravel()
    return df

def get_all_splits(df, regularizer, outcomes):
    data_splits = {}
    for outcome in outcomes:
        splits = {str(resampler): get_splits(df, outcome[0], regularizer, [resampler]) for resampler in resamplers}
        data_splits[outcome[1]] = splits
    return data_splits

def get_splits(df, y, regularizer = None, resamplers = None):
    x = df.values
    feature_labels = list(df.columns)
    loo = LeaveOneOut()
    splits = []
    for train, test in loo.split(x):
        split = {}
        xtrain, ytrain = x[train], y[train]
        xtest, ytest = x[test], y[test]
        if regularizer is not None:
            xtrain = regularizer.fit_transform(xtrain)
            xtest = regularizer.transform(xtest)
        for resampler in resamplers:
            if resampler is None:
                continue
            xtrain, ytrain = resampler.fit_resample(xtrain, ytrain)
        split['xtrain'] = xtrain
        split['xtest'] = xtest
        split['ytrain'] = ytrain
        split['ytest'] = ytest
        split['train_index'] = train
        split['test_index'] = test
        split['feature_labels'] = feature_labels
        splits.append(split)
    return splits

def cluster_features(db,
                  baseline_features = baseline_feature_file,
                  use_baseline_features = True,
                  top_features = 'data/clustering_results/toxicityClustering.csv',
                  use_top_features = True,
                  discrete_features = False,
                  cluster_names = ['kmeans_k=4','cluster_labels']):
    baseline = pd.read_csv(baseline_features, index_col = 'Dummy.ID').drop('Unnamed: 0', axis = 1)
    all_clusters = set(['manhattan_k=2','manhattan_k=3','manhattan_k=4',
                        'cluster_labels','hc_ward2','hc_ward4',
                        'FT','AR','TOX'])
    non_features = list(all_clusters - set(cluster_names))
    
    if use_baseline_features:
        cluster_names = cluster_names + list(baseline.drop(non_features, axis = 1, errors='ignore').columns)
    if 'T.category' in cluster_names:
        dist_clusters['T.category'] = dist_clusters['T.category'].apply(lambda x: int(x[1]))
        
    if isinstance(top_features, str):
        dist_clusters = pd.read_csv(top_features, index_col = 0)
        dist_clusters.index.rename('Dummy.ID', inplace = True)
        if use_top_features:
            cluster_names = cluster_names + list( dist_clusters.drop(non_features,axis=1, errors='ignore').columns)
        df = baseline.merge(dist_clusters, on=['Dummy.ID'])
    else:
        df = baseline
    ft = df.FT.values
    ar = df.AR.values
    tox = df.TOX.values
    to_drop = set(df.columns) - set(cluster_names)
    df = df.drop(to_drop, axis = 1, errors = 'ignore')
    if discrete_features:
        df = discretize_continuous_fields(df, 5)
    columns = df.columns
    for col in columns:
        if col in all_clusters:
            groups = set(df[col].values)
            for g in groups:
                col_name = col + '=' + str(g)
                df[col_name] = df[col].values == g
            df = df.drop(col, axis = 1)
    return df, ft, ar, tox

def test_classifiers(classifiers, 
                     db = None, 
                     log = False,
                     feature_params = {},
                     regularizer = QuantileTransformer(),
                     data_splits = None,
                     print_importances = False,
                    additional_features = None):

    result_template = {'cluster_names': copy(str(feature_params['cluster_names'])),
                       'Baseline': str(feature_params['use_baseline_features']),
                       'Top_features': str(feature_params['use_top_features']),
                       'Top_feature_file': str(feature_params['top_features']),
                      }

    if log:
        timestamp = datetime.fromtimestamp(time()).strftime('%Y_%m_%d_%H%M%S')
        f = open(Constants.toxicity_log_file_root + timestamp +'.txt', 'w', buffering = 1)
        def write(string):
            print(string)
            f.write(str(string)+'\n')
    else:
        write = lambda string: print(string)
        
    df, ft, ar, tox = cluster_features(db, **feature_params)
    if additional_features is not None:
        #should be tuple of attributes, organ_list (default none) to pass to patientset.to_dataframe
        df = db.to_dataframe(additional_features[0], df, additional_features[1])
    write(str(feature_params))
    outcomes = [(ft, 'feeding_tube'), (ar, 'aspiration'), (tox, 'toxicity')]
    data_splits = get_all_splits(df, regularizer, outcomes) if data_splits is None else data_splits
    print('splits finished')
    results = []
    for classifier in classifiers:
        write(classifier)
        for outcome in outcomes:
            data_split = data_splits[outcome[1]]
            for resampler_name, splits in data_split.items():
                try:
                    write(resampler_name)
                    auc, importances = presplit_roc_cv(classifier, splits)
                    write(outcome[1])
                    write(auc)
                    if importances is not None and print_importances:
                        write(importances)
                    write('\n')
                    result = copy(result_template)
                    result['classifier'] = str(classifier)
                    result['outcome'] = str(outcome[1])
                    result['resampler'] = str(resampler_name)
                    result['AUC'] = auc
                    results.append(result)
                except Exception as e:
                    print(e)
    if log:
        f.close()
    return results

In [9]:
def augmented_db(db = None, db_args = {}):
    if db is None:
        db = PatientSet(**db_args)
    db.toxicity = db.feeding_tubes + db.aspiration > 0
    return(db)

db = augmented_db()

F:\Skool\EVL_Research\CAMP-RT\PYTHON\Patient.py:359: RuntimeWarning: invalid value encountered in true_divide
  mean_tumor_distances /= tumor_volume
F:\Skool\EVL_Research\CAMP-RT\PYTHON\Patient.py:360: RuntimeWarning: invalid value encountered in true_divide
  tumor_position /= tumor_volume


100 [{0, 1}, {2}]
128 [{0}, {1, 2, 3, 4}]
notation not accounted for in lymph nodes: R3/R4
notation not accounted for in lymph nodes: L2/3
notation not accounted for in lymph nodes: R3/4
notation not accounted for in lymph nodes: R2/3
notation not accounted for in lymph nodes: R2-R4
notation not accounted for in lymph nodes: L2/3
notation not accounted for in lymph nodes: L2/3
notation not accounted for in lymph nodes: L2/3
notation not accounted for in lymph nodes: R2/3
notation not accounted for in lymph nodes: R2/3/4
notation not accounted for in lymph nodes: L2/3
notation not accounted for in lymph nodes: L2/3
notation not accounted for in lymph nodes: R2/3
notation not accounted for in lymph nodes: R2/3
notation not accounted for in lymph nodes: R2/3
10021 [{0, 1}, {2}]
error reading tumor volume for  10041
10074 [{0, 1}, {2}]
error reading tumor volume for  10091
error reading tumor volume for  10148
10191 [{0, 1}, {2}]


W0925 18:11:57.659312  3688 deprecation_wrapper.py:119] From F:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0925 18:11:57.661313  3688 deprecation_wrapper.py:119] From F:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0925 18:11:57.666313  3688 deprecation_wrapper.py:119] From F:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0925 18:11:57.675313  3688 deprecation_wrapper.py:119] From F:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0925 18:11:57.824322  3688 deprecation_wrapper.py:119] From F:\Anaconda\lib\site-packages\keras\opt


patient data loaded...



In [100]:
classifiers = [
#                    DecisionTreeClassifier(),
#                    DecisionTreeClassifier(criterion='entropy'),
#                    XGBClassifier(1, booster = 'gblinear'),
#                    XGBClassifier(3, booster = 'gblinear'),
#                    XGBClassifier(5, booster = 'gblinear'),
#                    XGBClassifier(),
#                    XGBClassifier(booster = 'dart'),
                    LogisticRegression(C = 1, solver = 'lbfgs', max_iter = 3000),
#                    MetricLearningClassifier(use_softmax = True),
#                    MetricLearningClassifier(
#                            resampler = under_sampling.OneSidedSelection()),
#                    MetricLearningClassifier(
#                            resampler = under_sampling.CondensedNearestNeighbour()),
#                    ExtraTreesClassifier(n_estimators = 200),
#                    RandomForestClassifier(n_estimators = 200, max_depth = 3),
#                    BayesWrapper(),
                   ]

resamplers = [
                  None,
#                  under_sampling.RandomUnderSampler(),
#                  over_sampling.RandomOverSampler(),
#                  under_sampling.InstanceHardnessThreshold(
#                          estimator = MetricLearningClassifier(),
#                          cv = 18),
#                  under_sampling.InstanceHardnessThreshold(cv = 18),
#                  over_sampling.SMOTE(),
#                  combine.SMOTEENN(),
#                  combine.SMOTETomek(),
#                  under_sampling.InstanceHardnessThreshold(),
#                  under_sampling.RepeatedEditedNearestNeighbours(),
#                  under_sampling.EditedNearestNeighbours(),
#                  under_sampling.CondensedNearestNeighbour(),
#                  under_sampling.OneSidedSelection(),
                  ]

cluster_root= 'data/clustering_results/'
feature_file_names = ['toxicityClustering', 'aspirationClustering', 
                      'feedingTubeClustering','toxicityClusteringWithNonspatial',
                      'combinedClustering', 'combinedClusteringWithNonspatial']
feature_file_names = ['combinedClusteringWithNonspatial']
#llop through all files of feature extracted from toxicity clustering
all_results = []
additional_features = None

run = lambda x: test_classifiers(classifiers, db, 
                                 log = True, 
                                 feature_params = x, 
                                 additional_features = additional_features)
do_test = lambda: all_results.extend(run(feature_params))

def try_features():
    for feature_file in feature_file_names:
        feature_params['top_features'] = cluster_root + feature_file + '.csv'
        do_test()

In [101]:
#the the unsupervised AUC scores
def print_cluster_results(clusters, name):
    print(name)
    for outcome in ['feeding_tubes', 'aspiration','toxicity']:
        print(outcome)
        tox = getattr(db, outcome)
        pred_outcome = BayesWrapper().fit_predict(clusters, tox.reshape(-1,1), True)[:,1]
        print(roc_auc_score(tox, pred_outcome))
    print()
    
baseline_clusters = pd.read_csv(baseline_feature_file, index_col='Dummy.ID')['manhattan_k=4'].values.reshape(-1,1)
print_cluster_results(baseline_clusters, 'baseline')

for feature_file in feature_file_names:
    file = cluster_root + feature_file + '.csv'
    spatial_clusters = pd.read_csv(file).cluster_labels.values.reshape(-1,1)
    print_cluster_results(spatial_clusters, feature_file)
    both_clusters = np.hstack([spatial_clusters, baseline_clusters])
    print_cluster_results(both_clusters, feature_file + '+baseline')

baseline
feeding_tubes
0.7258682328907048
aspiration
0.7525443442861297
toxicity
0.7402551381998583

combinedClusteringWithNonspatial
feeding_tubes
0.7589376915219612
aspiration
0.8569351555684792
toxicity
0.8038625088589653

combinedClusteringWithNonspatial+baseline
feeding_tubes
0.7886874361593463
aspiration
0.8828147717359698
toxicity
0.8416017009213324



In [102]:
#try out combinations of just clusters
feature_params = {
    'use_baseline_features': False,
    'top_features': None,
    'use_top_features': False,       
    'discrete_features': False,
    'cluster_names': ['manhattan_k=4']
}
do_test()
for cluster_combo in [['manhattan_k=4', 'cluster_labels'],['cluster_labels']]:
    feature_params['cluster_names'] = cluster_combo
    try_features()

{'use_baseline_features': False, 'top_features': None, 'use_top_features': False, 'discrete_features': False, 'cluster_names': ['manhattan_k=4']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
None
feeding_tube
0.5865679264555669


None
aspiration
0.49723756906077343


None
toxicity
0.6031183557760453


{'use_baseline_features': False, 'top_features': 'data/clustering_results/combinedClusteringWithNonspatial.csv', 'use_top_features': False, 'discrete_features': False, 'cluster_names': ['manhattan_k=4', 'cluster_labels']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                  

In [97]:
#baseline test
feature_params['use_baseline_features'] = True
for cluster_combo in [[],['manhattan_k=4']]:
    feature_params['cluster_names'] = cluster_combo
    do_test()

{'use_baseline_features': True, 'top_features': 'data/clustering_results/toxicityClustering.csv', 'use_top_features': False, 'discrete_features': False, 'cluster_names': []}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
None
feeding_tube
0.6404494382022471


None
aspiration
0.8493748182611224


None
toxicity
0.7888022678951098


{'use_baseline_features': True, 'top_features': 'data/clustering_results/toxicityClustering.csv', 'use_top_features': False, 'discrete_features': False, 'cluster_names': ['manhattan_k=4']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi

In [103]:
#test out adding clusters from each type (baseline + clusters)
feature_params['use_baseline_features'] = True
feature_params['use_top_features'] = False
feature_params['cluster_names'] = ['manhattan_k=4', 'cluster_labels']
try_features()

feature_params['cluster_names'] = ['cluster_labels']
try_features()

{'use_baseline_features': True, 'top_features': 'data/clustering_results/combinedClusteringWithNonspatial.csv', 'use_top_features': False, 'discrete_features': False, 'cluster_names': ['manhattan_k=4', 'cluster_labels']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
None
feeding_tube
0.6350868232890705


None
aspiration
0.8493748182611225


None
toxicity
0.7834868887313962


{'use_baseline_features': True, 'top_features': 'data/clustering_results/combinedClusteringWithNonspatial.csv', 'use_top_features': False, 'discrete_features': False, 'cluster_names': ['cluster_labels']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_sca

In [104]:
#now try just spatial features
feature_params['use_top_features'] = True
feature_params['use_baseline_features'] = False
feature_params['cluster_names']= ['cluster_labels']
try_features()

feature_params['use_baseline_features'] = True
try_features()

{'use_baseline_features': False, 'top_features': 'data/clustering_results/combinedClusteringWithNonspatial.csv', 'use_top_features': True, 'discrete_features': False, 'cluster_names': ['cluster_labels']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
None
feeding_tube
0.7456588355464759


None
aspiration
0.8371619656876998


None
toxicity
0.8125442948263643


{'use_baseline_features': True, 'top_features': 'data/clustering_results/combinedClusteringWithNonspatial.csv', 'use_top_features': True, 'discrete_features': False, 'cluster_names': ['cluster_labels']}
splits finished
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=N

In [49]:
#save it all
df = pd.DataFrame(all_results).sort_values(
        ['classifier',
         'outcome',
         'AUC',
         'resampler',
         'cluster_names',
         'Baseline'],
         kind = 'mergesort',
         ascending = False)
df.to_csv('data/toxcity_classification_tests_'
          + datetime.fromtimestamp(time()).strftime('%Y_%m_%d_%H%M%S')
          + '.csv', index = False)